## Installing Dependencies

In [ ]:
!pip install langchain==0.3.27 langchain-openai==0.3.30 langgraph==0.6.5 langchain-mcp-adapters==0.1.9 mcp==1.12.4 --quiet

## Building a MCP Server for the Finance Department

This section defines a **FastMCP** server named `FinanceServer` that runs on **port `8010`** and is started with **`mcp.run(transport="streamable-http")`**.

It registers two tools via `@mcp.tool()`:

- `generate_invoice(customer_id: str) -> dict`  
  Returns a stubbed invoice payload, e.g.  
  `{"invoice_id": "INV-1001", "customer_id": "<id>", "amount": 2500, "status": "generated"}`

- `get_budget_summary(department: str) -> dict`  
  Returns a simple budget snapshot, e.g.  
  `{"department": "<dept>", "budget": 100000, "spent": 65432, "remaining": 34568}`

The server’s `FastMCP(...)` constructor includes a short instruction string and the explicit port.


In [ ]:
%%writefile finance_mcp_server.py

from mcp.server.fastmcp import FastMCP

# Server with instructions
mcp = FastMCP(
    name="FinanceServer",
    instructions="""
        This server provides finance-related tools.
        - Call generate_invoice(customer_id) to generate a new invoice for a customer.
        - Call get_budget_summary(department) to retrieve the budget overview of any department.
    """,
    port=8010
)

@mcp.tool()
def generate_invoice(customer_id: str) -> dict:
    return {
        "invoice_id": "INV-1001",
        "customer_id": customer_id,
        "amount": 2500,
        "status": "generated"
    }

@mcp.tool()
def get_budget_summary(department: str) -> dict:
    return {
        "department": department,
        "budget": 100000,
        "spent": 65432,
        "remaining": 34568
    }

if __name__ == "__main__":
    print("Starting Finance MCP Server...")
    mcp.run(transport="streamable-http")


Overwriting finance_mcp_server.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Deploying the MCP Server for the Finance Department

The Finance server is launched as a background process in the notebook server:
- Starts FinanceServer on ` http://localhost:8010/mcp` using the `streamable-http` transport.
- Logs are written to `finance_server_output.log`.


In [ ]:
!nohup python /content/finance_mcp_server.py > finance_server_output.log 2>&1 &

![](https://i.imgur.com/8n4J8qV.png)

## Building a MCP Server for the HR Department

This section defines a **FastMCP** server named `HRServer` that runs on **port `8011`** and is started with **`mcp.run(transport="streamable-http")`**.

It registers two tools via `@mcp.tool()`:

- `get_employee_details(employee_id: str) -> dict`  
  Returns a stubbed employee profile, e.g.  
  `{"employee_id": "<id>", "name": "Alice Johnson", "role": "Software Engineer", "department": "Tech"}`

- `check_leave_balance(employee_id: str) -> dict`  
  Returns leave information, e.g.  
  `{"employee_id": "<id>", "leave_balance": 12}`

Like the Finance server, `FastMCP(...)` includes instructions and the explicit port.


In [ ]:
%%writefile hr_mcp_server.py

from mcp.server.fastmcp import FastMCP

# Server with instructions
mcp = FastMCP(
    name="HRServer",
    instructions="""
        This server provides human resources tools.
        - Call get_employee_details(employee_id) to fetch an employee's information.
        - Call check_leave_balance(employee_id) to view the available leave balance for an employee.
    """,
    port=8011
)

@mcp.tool()
def get_employee_details(employee_id: str) -> dict:
    return {
        "employee_id": employee_id,
        "name": "Alice Johnson",
        "role": "Software Engineer",
        "department": "Tech"
    }

@mcp.tool()
def check_leave_balance(employee_id: str) -> dict:
    return {
        "employee_id": employee_id,
        "leave_balance": 12
    }

if __name__ == "__main__":
    print("Starting HR MCP Server...")
    mcp.run(transport="streamable-http")

Writing hr_mcp_server.py


## Deploying the MCP Server for the HR Department

The HR server is launched as a background process in the notebook server:
- Starts HRServer on ` http://localhost:8011/mcp` using the `streamable-http` transport.
- Logs are written to `hr_server_output.log`.

In [ ]:
!nohup python /content/hr_mcp_server.py > hr_server_output.log 2>&1 &

![](https://i.imgur.com/sHdQjL9.png)

## Building the Client Agent

This section creates a client agent that connects to both servers:

- **MCP client:**  
  ```python
  client = MultiServerMCPClient({
      "finance": {"url": "http://localhost:8010/mcp", "transport": "streamable_http"},
      "hr":      {"url": "http://localhost:8011/mcp", "transport": "streamable_http"}
  })
  ```
- **LLM:**  
  ```python
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
  ```

- **Tool discovery:**  
  ```python
  tools = await client.get_tools()
  print("Discovered tools:", [tool.name for tool in tools])
  ```

- **Agent:**  
  ```python
  agent = create_react_agent(model=llm, tools=tools)
  ```

This setup produces a single agent capable of invoking tools from **both** `FinanceServer` and `HRServer` using MCP.

In [ ]:
%%writefile client_agent.py

from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
import asyncio
from dotenv import load_dotenv

load_dotenv()
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0,openai_api_key="your_api_key")

async def main():
    client = MultiServerMCPClient({
        "finance": {
            "url": "http://localhost:8010/mcp",
            "transport": "streamable_http"
        },
        "hr": {
            "url": "http://localhost:8011/mcp",
            "transport": "streamable_http"
        }
    })

    tools = await client.get_tools()
    print("Discovered tools:", [tool.name for tool in tools])

    agent = create_react_agent(model=llm, tools=tools)

    # Sample queries
    print('\nUsing capabilities from Finance - Invoice Generation:')
    result_1 = await agent.ainvoke({"messages": [{"role": "user", "content": "Can you generate an invoice for customer 456?"}]})
    print(result_1['messages'][-1].content)

    print('\nUsing capabilities from HR - Leave Balance:')
    result_2 = await agent.ainvoke({"messages": [{"role": "user", "content": "Show me the leave balance for employee 456"}]})
    print(result_2['messages'][-1].content)

    print('\nUsing capabilities from Finance - Budget Planning:')
    result_3 = await agent.ainvoke({"messages": [{"role": "user", "content": "What is the tech department’s budget summary?"}]})
    print(result_3['messages'][-1].content)

if __name__ == "__main__":
    asyncio.run(main())

Writing client_agent.py


## Running the Client Agent

![](https://i.imgur.com/S2vfh6c.png)